In [1]:
import os

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import librosa
import numpy as np

from sklearn.utils import shuffle
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import imageio
import glob
from collections import Counter
from matplotlib.pyplot import hist

import matplotlib.pyplot as plt

import tensorflow as tf

Paths, Spectrogram Variables

In [2]:
path = 'birdclef/'
path_train_meta = path + 'train_metadata.csv'
path_train_soundscape = path + 'train_soundscape_labels.csv'
input_dir = path + 'train_short_audio/'
output_dir = 'working/melspectrogram_dataset/'

SAMPLE_RATE = 32000
SIGNAL_LENGTH = 5 # seconds
SPEC_SHAPE = (48, 128) # height x width
FMIN = 500
FMAX = 12500
dimension = (48, 128, 1)
SPEC_SHAPE = (48, 128)
random_seed = 666

model_name = 'models/TrainAudioOnly.h5'

Prepare List of Birds which are trained

In [3]:
#soundscape ids controls which birds are taken from dataset 
soundscapes_ids = [28933, 42907]
soundscapes = pd.read_csv('birdclef/train_soundscape_labels.csv',)
soundscapes = soundscapes.query('audio_id in @soundscapes_ids')
names = pd.unique(soundscapes['birds'])

bird_list = []
for i in range (0, len(names)):
    split = names[i].split(' ')
    bird_list = np.append(bird_list, split)
    
bird_list = np.unique(bird_list)
bird_list = list(bird_list)
bird_list.remove('nocall')

Load Data

In [4]:
#bird_list = ['amerob', 'blujay', 'cangoo', 'gockin', 'norcar', 'rewbla', 'sonspa', 'swaspa']
data = []
labels = []

distribution = [(bird, 0) for bird in bird_list]

#specify the maximum of how many 5 second slides are taken to the train dataset for one bird
num_spec_slices = 1500


csv = pd.read_csv('birdclef/train_metadata.csv',)
csv = csv.query('rating>=4')
csv = csv.query('primary_label in @bird_list')

csv_array = np.array(csv)

old_bird_name = ""
counter = 0

with tqdm(total=len(bird_list)*num_spec_slices) as pbar:
    #Go through lines of csv file
    for i in range(0, len(csv)):      
        bird_name = csv_array[i][0]
        filename = csv_array[i][9][:-4]
        index = bird_list.index(bird_name)
        
        if bird_name != old_bird_name:
            counter = 0
        old_bird_name = bird_name
        
        if counter >= num_spec_slices:
            continue
        
        #Go through working directory and select 5s slices which belongs to actual audio file from csv
        for spec in glob.glob(output_dir + bird_name + "/" + filename + "*"):
            if counter >= num_spec_slices:
                break           
            im = imageio.imread(spec)
            im = np.resize(im, dimension)
            data.append(im)
            zeros = np.zeros(len(bird_list))
            zeros[index] = 1
            labels.append(zeros)
            counter += 1
            pbar.update(1) 

 84%|████████▎ | 16291/19500 [00:19<00:03, 819.89it/s] 


In [5]:
data = shuffle(data, random_state=random_seed)
labels = shuffle(labels, random_state=random_seed)

In [6]:
# Make sure your experiments are reproducible
tf.random.set_seed(random_seed)

# Build a simple model as a sequence of  convolutional blocks.
# Each block has the sequence CONV --> RELU --> BNORM --> MAXPOOL.
# Finally, perform global average pooling and add 2 dense layers.
# The last layer is our classification layer and is softmax activated.
# (Well it's a multi-label task so sigmoid might actually be a better choice)
model = tf.keras.Sequential([
    
    # First conv block
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', 
                           input_shape=(SPEC_SHAPE[0], SPEC_SHAPE[1], 1)),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
 
        
    # Second conv block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Third conv block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)), 
    

    
    # Global pooling instead of flatten()
    tf.keras.layers.Flatten(), 
    
    # Dense block
    tf.keras.layers.Dense(256, activation='relu'),   
    tf.keras.layers.Dropout(0.5),      
    tf.keras.layers.Dense(128, activation='relu'),   
    tf.keras.layers.Dropout(0.5),  
    
    # Classification layer
    tf.keras.layers.Dense(len(bird_list), activation='softmax')
])
print('MODEL HAS {} PARAMETERS.'.format(model.count_params()))

MODEL HAS 500733 PARAMETERS.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 126, 16)       160       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 126, 16)       64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 124, 16)       2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 124, 16)       64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 62, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 60, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 60, 32)        1

Hyperparameters

In [8]:
lr = 0.001
label_smoothing = 0.1
batch_size = 32

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
              metrics=['accuracy'])

In [10]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=1, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              verbose=1,
                                              patience=2),
             tf.keras.callbacks.ModelCheckpoint(filepath=model_name, 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

In [14]:
#model = tf.keras.models.load_model(model_name)
model.fit(np.array(data[:int(len(data)*0.9)]), 
          np.array(labels[:int(len(data)*0.9)]),
          batch_size=batch_size,
          validation_split=0.2,
          callbacks=callbacks,
          workers=16,
          epochs=25)

Epoch 1/25
367/367 [==============================] - 37s 99ms/step - loss: 2.4487 - accuracy: 0.2139 - val_loss: 2.0452 - val_accuracy: 0.3737
Epoch 2/25
367/367 [==============================] - 36s 98ms/step - loss: 2.0051 - accuracy: 0.3944 - val_loss: 1.7066 - val_accuracy: 0.5343
Epoch 3/25
367/367 [==============================] - 36s 98ms/step - loss: 1.7894 - accuracy: 0.5077 - val_loss: 1.6089 - val_accuracy: 0.6055
Epoch 4/25
367/367 [==============================] - 36s 98ms/step - loss: 1.6472 - accuracy: 0.5819 - val_loss: 1.4905 - val_accuracy: 0.6655
Epoch 5/25
367/367 [==============================] - 36s 98ms/step - loss: 1.5487 - accuracy: 0.6305 - val_loss: 1.3876 - val_accuracy: 0.7017
Epoch 6/25
367/367 [==============================] - 36s 98ms/step - loss: 1.4460 - accuracy: 0.6833 - val_loss: 1.4272 - val_accuracy: 0.6904

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/25
367/367 [==============================] - 3

In [ ]:
def prediction(arr):
    return bird_list[np.argmax(arr)]

Prediction method: 
Insert index of train dataset and it predicts the bird 

In [ ]:
def eval_prediction(index=0):
    input_bird = np.resize(data[index], (1, 48, 128, 1))
    result = list(zip(bird_list, model.predict(input_bird)[0]*100))
    return (prediction(labels[index]), result)

eval_prediction(200)

Plot the distribution of the bird dataset

In [ ]:
a = np.array(labels)
unique, counts = np.unique(a, return_counts=True, axis=0)
X = []
for i in range(0, len(unique)):
    X.append(prediction(unique[i]))


plt.bar([x for x in range(0, len(counts))], counts, tick_label=X)
plt.xticks(rotation=70)

plt.show()